In [1]:
import numpy as np
import cv2
from glob import glob
from matplotlib import pyplot as plt
import matplotlib.patches as patches
import pandas as pd
from pathlib import Path
import re
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from models import ArcFaceModel
from losses import softmax_loss
import dataset
import tensorflow as tf
import os
import logging
import tensorflow.keras.backend as K
from tensorflow.keras.models import Model, load_model
from layers import ArcMarginPenaltyLogists
from tqdm import tqdm
from utils import l2_norm
import logging

tf.get_logger().setLevel(logging.ERROR)
os.environ["CUDA_VISIBLE_DEVICES"]="1"

# Load Model

In [16]:
### MS1M dataset

batch_size = 128 # Initially 128
input_size = 112
embd_shape = 512
head_type = 'CurHead' # ''ArcHead', CosHead', 'SphereHead'
# Backbones w/ pretrained weights:
#     MobileNet, MobileNetV2, InceptionResNetV2, InceptionV3, ResNet50, ResNet50V2, ResNet101V2, NASNetLarge, NASNetMobile, Xception
#     But if you are trying to use NasNet, please check this issue first: https://github.com/keras-team/keras-applications/issues/78
#         We manually download the weight file and explicitly load it in models.py file
# Backbones w/o pretrained weights:
#     MobileNetV3Large, MobileNetV3Small, EfficientNetLite0~6, EfficientNetB0~7
backbone_type = 'EfficientNetB6' 
w_decay=5e-4
num_classes = 85742 
dataset_len = 5822653 
base_lr = 0.01 # initially 0.01
epochs = 20
save_steps = 1000
train_size = int(0.8 * dataset_len)
print("train_size: ",train_size)
steps_per_epoch = train_size // batch_size
print("steps_per_epoch: ",steps_per_epoch)
val_size = dataset_len - train_size
print("val_size: ",val_size)
validation_steps = val_size // batch_size
print("validation_steps: ",validation_steps)
steps = 1
is_ccrop=False
binary_img=True
is_Adam = False
projection_head = False  # True
dgx = True

version = "Check"

if dgx:
    base_dir = "/raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/"
    if projection_head:
        save_name = f'ms1m_{backbone_type}_{head_type}_ProjectionHead_check/{version}/*'
    else:
        save_name = f'ms1m_{backbone_type}_{head_type}_check/{version}/*'
else:
    base_dir = "/hd/honghee/models/checkpoints/w_tfidentity/"
    save_name = f'ms1m_{backbone_type}_{head_type}_check/{version}/*'
    
# collect loss in checkpoints
# file_list = []
# for files in glob(base_dir+save_name):
#     if not files.split('/')[-1].split('l_')[-1].split('.ckpt')[0] == 'checkpoint':
#         loss = float( files.split('/')[-1].split('l_')[-1].split('.ckpt')[0] )
#     file_list.append( loss  )
# file_list.sort()

# load_file_name = []
# for files in glob(base_dir+save_name):
#     if files.split('/')[-1].split('l_')[-1].split('.ckpt')[0] == 'checkpoint':
#         pass
#     elif file_list[0] == float( files.split('/')[-1].split('l_')[-1].split('.ckpt')[0] ) and files.split('/')[-1].split('l_')[-1].split('.ckpt')[-1]!='.index':
#         load_file_name = files
# best_checkpoint = load_file_name.split('.data')[0]
# print("best checkpoint: ",best_checkpoint)

train_size:  4658122
steps_per_epoch:  36391
val_size:  1164531
validation_steps:  9097


In [17]:
best_checkpoint = "/raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_CurHead_check/SGD/e_14_l_0.45593419671058655.ckpt"
print("best checkpoint: ",best_checkpoint)
weight_file = best_checkpoint
model = ArcFaceModel(size=input_size,
                         backbone_type=backbone_type,
                         training=False)
model.load_weights(weight_file)
model.summary()

best checkpoint:  /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_CurHead_check/SGD/e_14_l_0.45593419671058655.ckpt
Model: "arcface_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_image (InputLayer)     [(None, 112, 112, 3)]     0         
_________________________________________________________________
efficientnetb6 (Functional)  (None, 4, 4, 2304)        40960136  
_________________________________________________________________
OutputLayer (Functional)     (None, 512)               18886144  
Total params: 59,846,280
Trainable params: 59,610,584
Non-trainable params: 235,696
_________________________________________________________________


# For IJBC

In [2]:
import os
import pickle

import matplotlib
import pandas as pd

matplotlib.use('Agg')
import matplotlib.pyplot as plt
import timeit
import sklearn
import argparse
from sklearn.metrics import roc_curve, auc

from pathlib import Path
import sys
import warnings

import cv2
import numpy as np
from skimage import transform as trans

In [32]:
meta_path = "/raid/workspace/jbpark/IJB-C/IJBC/meta/"

enroll_df = pd.read_csv("/raid/workspace/jihyun/IJBC/IJB/IJB-C/protocols/test1/enroll_templates.csv")
verif_temp_df = pd.read_csv("/raid/workspace/jihyun/IJBC/IJB/IJB-C/protocols/test1/verif_templates.csv")
image_df = pd.concat([enroll_df, verif_temp_df])
meta = pd.read_csv("/raid/workspace/jihyun/IJBC/IJB/IJB-C/protocols/ijbc_metadata_with_age.csv")
meta = meta[["SUBJECT_ID","FILENAME","SIGHTING_ID"]]
veri_df = pd.merge(image_df, meta, on=["FILENAME", "SUBJECT_ID"])

In [33]:
def read_template_media_list(df):
    # ijb_meta = np.loadtxt(path, dtype=str)
    ijb_meta = df.values
    templates = ijb_meta[:, 0].astype(np.int)
    medias = ijb_meta[:, -1].astype(np.int)
    return templates, medias

In [5]:
def read_template_pair_list(path):
    # pairs = np.loadtxt(path, dtype=str)
    pairs = pd.read_csv(path, sep=' ', header=None).values
    # print(pairs.shape)
    # print(pairs[:, 0].astype(np.int))
    t1 = pairs[:, 0].astype(np.int)
    t2 = pairs[:, 1].astype(np.int)
    label = pairs[:, 2].astype(np.int)
    return t1, t2, label

# STEP1:  Load Meta Data

In [34]:
# =============================================================
# load image and template relationships for template feature embedding
# tid --> template id,  mid --> media id
# format:
#           image_name tid mid
# =============================================================
start = timeit.default_timer()
templates, medias = read_template_media_list(veri_df)
stop = timeit.default_timer()
print('Time: %.2f s. ' % (stop - start))

Time: 0.18 s. 


In [35]:
templates

array([     1,      1,      1, ..., 187953, 187954, 187955])

In [36]:
medias

array([ 69544,   3720,      3, ..., 111105, 111105, 111105])

In [37]:
def template_encoding(templates, medias, img_norm_feats):
# def template_encoding(templates, medias, img_norm_feats):
    # ==========================================================
    # 1. face image --> l2 normalization.
    # 2. compute media encoding.
    # 3. compute template encoding.
    # 4. save template features.
    # ==========================================================
    print('==> compute template-level feature encoding.')

    uq_temp = np.unique(templates)
    num_temp = len(uq_temp)
    tmp_feats = np.empty((num_temp, 512))

    for c, uqt in enumerate(uq_temp):
        (ind_t,) = np.where(templates == uqt)
#         face_norm_feats = img_norm_feats[ind_t]
        faces_media = medias[ind_t]
        uqm, counts = np.unique(faces_media, return_counts=True)
        media_norm_feats = []

        for u,ct in zip(uqm, counts):
            (ind_m,) = np.where(faces_media == u)
            if ct < 2:
                media_norm_feats += [face_norm_feats[ind_m]]
            else:
                media_norm_feats += [np.sum(face_norm_feats[ind_m], 0, keepdims=True)]

        media_norm_feats = np.array(media_norm_feats)
        media_norm_feats = media_norm_feats / np.sqrt(np.sum(media_norm_feats ** 2, -1, keepdims=True))
        template_norm_feats = np.sum(media_norm_feats, 0)
        tmp_feats[c] = template_norm_feats
        if c % 500 == 0:
            print('-> finish encoding {}/{} templates.'.format(c, num_temp))
    return tmp_feats


In [38]:
# =============================================================
# load template pairs for template-to-template verification
# tid : template id,  label : 1/0
# format:
#           tid_1 tid_2 label
# =============================================================
start = timeit.default_timer()
p1, p2, label = read_template_pair_list(meta_path +"ijbc_template_pair_label.txt")
stop = timeit.default_timer()
print('Time: %.2f s. ' % (stop - start))

Time: 2.95 s. 


In [39]:
p1

array([     1,      1,      1, ..., 171707, 171707, 171707])

In [56]:
np.unique(p1).shape

(3531,)

In [40]:
p2

array([ 11065,  11066,  11067, ..., 183140, 182146, 185794])

In [57]:
np.unique(p2).shape

(19593,)

In [41]:
label

array([1, 1, 1, ..., 0, 0, 0])

In [58]:
len(label)

15658489

# STEP2: Get Image Features

In [13]:
def get_embed(rimg, landmark, input_size):
    image_size = (input_size,input_size)

#     assert landmark.shape[0] == 68 or landmark.shape[0] == 5
#     assert landmark.shape[1] == 2
#     if landmark.shape[0] == 68:
#         landmark5 = np.zeros((5, 2), dtype=np.float32)
#         landmark5[0] = (landmark[36] + landmark[39]) / 2
#         landmark5[1] = (landmark[42] + landmark[45]) / 2
#         landmark5[2] = landmark[30]
#         landmark5[3] = landmark[48]
#         landmark5[4] = landmark[54]
#     else:
#         landmark5 = landmark
#     tform = trans.SimilarityTransform()
#     src = np.array([
#         [30.2946, 51.6963],
#         [65.5318, 51.5014],
#         [48.0252, 71.7366],
#         [33.5493, 92.3655],
#         [62.7299, 92.2041]], dtype=np.float32)
#     src[:, 0] += 8.0
#     tform.estimate(landmark5, src)
#     M = tform.params[0:2, :]
#     img = cv2.warpAffine(rimg,
#                          M, (image_size[1], image_size[0]),
#                          borderValue=0.0)

    img = cv2.resize(rimg, dsize=(image_size[1], image_size[0]))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_flip = np.fliplr(img)
#     img = np.transpose(img, (2, 0, 1))  # 3*112*112, RGB
#     img_flip = np.transpose(img_flip, (2, 0, 1))
    input_blob = np.zeros((2, image_size[1], image_size[0], 3), dtype=np.uint8)
    input_blob[0] = img
    input_blob[1] = img_flip
    return input_blob

In [14]:
def get_image_feature(img_path, files_list, model):
    batch_size = 128

    files = files_list
    print('files:', len(files))
    rare_size = len(files) % batch_size
    faceness_scores = []
    batch = 0
    img_feats = np.empty((len(files), 1024), dtype=np.float32)

    batch_data = np.empty((2 * batch_size, input_size, input_size, 3))
    for img_index, each_line in enumerate(files[:len(files) - rare_size]):
        name_lmk_score = each_line.strip().split(' ')
        img_name = os.path.join(img_path, name_lmk_score[0])
        img = cv2.imread(img_name)
        lmk = np.array([float(x) for x in name_lmk_score[1:-1]],
                       dtype=np.float32)
        lmk = lmk.reshape((5, 2))
        input_blob = get_embed(img, lmk, input_size)

        batch_data[2 * (img_index - batch * batch_size)][:] = input_blob[0]
        batch_data[2 * (img_index - batch * batch_size) + 1][:] = input_blob[1]
        if (img_index + 1) % batch_size == 0:
            print('batch', batch)
            img_feats[batch * batch_size:batch * batch_size +
                      batch_size][:] = model.predict(batch_data).reshape([batch_size,2*512])
            batch += 1
        faceness_scores.append(name_lmk_score[-1])

    batch_data = np.empty((2 * rare_size, input_size, input_size, 3))
    
    # batch size로 안나눠지는 남는 것들 처리
    for img_index, each_line in enumerate(files[len(files) - rare_size:]):
        name_lmk_score = each_line.strip().split(' ')
        img_name = os.path.join(img_path, name_lmk_score[0])
        img = cv2.imread(img_name)
        lmk = np.array([float(x) for x in name_lmk_score[1:-1]],
                       dtype=np.float32)
        lmk = lmk.reshape((5, 2))
        input_blob = get_embed(img, lmk, input_size)
        batch_data[2 * img_index][:] = input_blob[0]
        batch_data[2 * img_index + 1][:] = input_blob[1]
        if (img_index + 1) % rare_size == 0:
            print('batch', batch)
            img_feats[len(files) - rare_size:][:] = model.predict(batch_data).reshape([rare_size,2*512])
            batch += 1
        faceness_scores.append(name_lmk_score[-1])
    faceness_scores = np.array(faceness_scores).astype(np.float32)
    # img_feats = np.ones( (len(files), 1024), dtype=np.float32) * 0.01
    # faceness_scores = np.ones( (len(files), ), dtype=np.float32 )
    return img_feats, faceness_scores


In [15]:
# =============================================================
# load image features
# format:
#           img_feats: [image_num x feats_dim] (227630, 512)
# =============================================================
start = timeit.default_timer()
image_path = "/raid/workspace/jbpark/IJB-C"
img_path = '%s/veri_crops_v2' % image_path
print(img_path)
img_list_path = meta_path +"ijbc_name_5pts_score.txt"
img_list = open(img_list_path)
files = img_list.readlines()
# files_list = divideIntoNstrand(files, rank_size)
files_list = files

/raid/workspace/jbpark/IJB-C/veri_crops_v2


In [18]:
# img_feats
# for i in range(rank_size):
img_feats, faceness_scores = get_image_feature(img_path, files_list, model)
stop = timeit.default_timer()
print('Time: %.2f s. ' % (stop - start))
print('Feature Shape: ({} , {}) .'.format(img_feats.shape[0],
                                          img_feats.shape[1]))


files: 469375
batch 0
batch 1
batch 2
batch 3
batch 4
batch 5
batch 6
batch 7
batch 8
batch 9
batch 10
batch 11
batch 12
batch 13
batch 14
batch 15
batch 16
batch 17
batch 18
batch 19
batch 20
batch 21
batch 22
batch 23
batch 24
batch 25
batch 26
batch 27
batch 28
batch 29
batch 30
batch 31
batch 32
batch 33
batch 34
batch 35
batch 36
batch 37
batch 38
batch 39
batch 40
batch 41
batch 42
batch 43
batch 44
batch 45
batch 46
batch 47
batch 48
batch 49
batch 50
batch 51
batch 52
batch 53
batch 54
batch 55
batch 56
batch 57
batch 58
batch 59
batch 60
batch 61
batch 62
batch 63
batch 64
batch 65
batch 66
batch 67
batch 68
batch 69
batch 70
batch 71
batch 72
batch 73
batch 74
batch 75
batch 76
batch 77
batch 78
batch 79
batch 80
batch 81
batch 82
batch 83
batch 84
batch 85
batch 86
batch 87
batch 88
batch 89
batch 90
batch 91
batch 92
batch 93
batch 94
batch 95
batch 96
batch 97
batch 98
batch 99
batch 100
batch 101
batch 102
batch 103
batch 104
batch 105
batch 106
batch 107
batch 108
batch 

batch 829
batch 830
batch 831
batch 832
batch 833
batch 834
batch 835
batch 836
batch 837
batch 838
batch 839
batch 840
batch 841
batch 842
batch 843
batch 844
batch 845
batch 846
batch 847
batch 848
batch 849
batch 850
batch 851
batch 852
batch 853
batch 854
batch 855
batch 856
batch 857
batch 858
batch 859
batch 860
batch 861
batch 862
batch 863
batch 864
batch 865
batch 866
batch 867
batch 868
batch 869
batch 870
batch 871
batch 872
batch 873
batch 874
batch 875
batch 876
batch 877
batch 878
batch 879
batch 880
batch 881
batch 882
batch 883
batch 884
batch 885
batch 886
batch 887
batch 888
batch 889
batch 890
batch 891
batch 892
batch 893
batch 894
batch 895
batch 896
batch 897
batch 898
batch 899
batch 900
batch 901
batch 902
batch 903
batch 904
batch 905
batch 906
batch 907
batch 908
batch 909
batch 910
batch 911
batch 912
batch 913
batch 914
batch 915
batch 916
batch 917
batch 918
batch 919
batch 920
batch 921
batch 922
batch 923
batch 924
batch 925
batch 926
batch 927
batch 928


batch 1590
batch 1591
batch 1592
batch 1593
batch 1594
batch 1595
batch 1596
batch 1597
batch 1598
batch 1599
batch 1600
batch 1601
batch 1602
batch 1603
batch 1604
batch 1605
batch 1606
batch 1607
batch 1608
batch 1609
batch 1610
batch 1611
batch 1612
batch 1613
batch 1614
batch 1615
batch 1616
batch 1617
batch 1618
batch 1619
batch 1620
batch 1621
batch 1622
batch 1623
batch 1624
batch 1625
batch 1626
batch 1627
batch 1628
batch 1629
batch 1630
batch 1631
batch 1632
batch 1633
batch 1634
batch 1635
batch 1636
batch 1637
batch 1638
batch 1639
batch 1640
batch 1641
batch 1642
batch 1643
batch 1644
batch 1645
batch 1646
batch 1647
batch 1648
batch 1649
batch 1650
batch 1651
batch 1652
batch 1653
batch 1654
batch 1655
batch 1656
batch 1657
batch 1658
batch 1659
batch 1660
batch 1661
batch 1662
batch 1663
batch 1664
batch 1665
batch 1666
batch 1667
batch 1668
batch 1669
batch 1670
batch 1671
batch 1672
batch 1673
batch 1674
batch 1675
batch 1676
batch 1677
batch 1678
batch 1679
batch 1680

batch 2335
batch 2336
batch 2337
batch 2338
batch 2339
batch 2340
batch 2341
batch 2342
batch 2343
batch 2344
batch 2345
batch 2346
batch 2347
batch 2348
batch 2349
batch 2350
batch 2351
batch 2352
batch 2353
batch 2354
batch 2355
batch 2356
batch 2357
batch 2358
batch 2359
batch 2360
batch 2361
batch 2362
batch 2363
batch 2364
batch 2365
batch 2366
batch 2367
batch 2368
batch 2369
batch 2370
batch 2371
batch 2372
batch 2373
batch 2374
batch 2375
batch 2376
batch 2377
batch 2378
batch 2379
batch 2380
batch 2381
batch 2382
batch 2383
batch 2384
batch 2385
batch 2386
batch 2387
batch 2388
batch 2389
batch 2390
batch 2391
batch 2392
batch 2393
batch 2394
batch 2395
batch 2396
batch 2397
batch 2398
batch 2399
batch 2400
batch 2401
batch 2402
batch 2403
batch 2404
batch 2405
batch 2406
batch 2407
batch 2408
batch 2409
batch 2410
batch 2411
batch 2412
batch 2413
batch 2414
batch 2415
batch 2416
batch 2417
batch 2418
batch 2419
batch 2420
batch 2421
batch 2422
batch 2423
batch 2424
batch 2425

batch 3080
batch 3081
batch 3082
batch 3083
batch 3084
batch 3085
batch 3086
batch 3087
batch 3088
batch 3089
batch 3090
batch 3091
batch 3092
batch 3093
batch 3094
batch 3095
batch 3096
batch 3097
batch 3098
batch 3099
batch 3100
batch 3101
batch 3102
batch 3103
batch 3104
batch 3105
batch 3106
batch 3107
batch 3108
batch 3109
batch 3110
batch 3111
batch 3112
batch 3113
batch 3114
batch 3115
batch 3116
batch 3117
batch 3118
batch 3119
batch 3120
batch 3121
batch 3122
batch 3123
batch 3124
batch 3125
batch 3126
batch 3127
batch 3128
batch 3129
batch 3130
batch 3131
batch 3132
batch 3133
batch 3134
batch 3135
batch 3136
batch 3137
batch 3138
batch 3139
batch 3140
batch 3141
batch 3142
batch 3143
batch 3144
batch 3145
batch 3146
batch 3147
batch 3148
batch 3149
batch 3150
batch 3151
batch 3152
batch 3153
batch 3154
batch 3155
batch 3156
batch 3157
batch 3158
batch 3159
batch 3160
batch 3161
batch 3162
batch 3163
batch 3164
batch 3165
batch 3166
batch 3167
batch 3168
batch 3169
batch 3170

# STEP3: Get Template Feature 

In [42]:
def image2template_feature(img_feats=None, templates=None, medias=None):
    # ==========================================================
    # 1. face image feature l2 normalization. img_feats:[number_image x feats_dim]
    # 2. compute media feature.
    # 3. compute template feature.
    # ==========================================================
    unique_templates = np.unique(templates)
    template_feats = np.zeros((len(unique_templates), img_feats.shape[1]))
    for count_template, uqt in enumerate(unique_templates):

        (ind_t,) = np.where(templates == uqt)
        face_norm_feats = img_feats[ind_t]
        face_medias = medias[ind_t]
        unique_medias, unique_media_counts = np.unique(face_medias,
                                                       return_counts=True)
        media_norm_feats = []
        for u, ct in zip(unique_medias, unique_media_counts):
            (ind_m,) = np.where(face_medias == u)
            if ct == 1:
                media_norm_feats += [face_norm_feats[ind_m]]
            else:  # image features from the same video will be aggregated into one feature
                media_norm_feats += [
                    np.mean(face_norm_feats[ind_m], axis=0, keepdims=True)
                ]
        media_norm_feats = np.array(media_norm_feats)
        # media_norm_feats = media_norm_feats / np.sqrt(np.sum(media_norm_feats ** 2, -1, keepdims=True))
        template_feats[count_template] = np.sum(media_norm_feats, axis=0)
        if count_template % 2000 == 0:
            print('Finish Calculating {} template features.'.format(
                count_template))
    # template_norm_feats = template_feats / np.sqrt(np.sum(template_feats ** 2, -1, keepdims=True))
    template_norm_feats = sklearn.preprocessing.normalize(template_feats)
    # print(template_norm_feats.shape)
    return template_norm_feats, unique_templates

In [59]:
# =============================================================
# compute template features from image features.
# =============================================================
start = timeit.default_timer()
# ==========================================================
# Norm feature before aggregation into template feature?
# Feature norm from embedding network and faceness score are able to decrease weights for noise samples (not face).
# ==========================================================
# 1. FaceScore （Feature Norm）
# 2. FaceScore （Detector）
use_flip_test = False
use_norm_score = False
use_detector_score = False

if use_flip_test:
    # concat --- F1
    # img_input_feats = img_feats
    # add --- F2
    img_input_feats = img_feats[:, 0:img_feats.shape[1] //
                                     2] + img_feats[:, img_feats.shape[1] // 2:]
else:
    img_input_feats = img_feats[:, 0:img_feats.shape[1] // 2]

if use_norm_score:
    img_input_feats = img_input_feats
else:
    # normalise features to remove norm information
    img_input_feats = img_input_feats / np.sqrt(
        np.sum(img_input_feats ** 2, -1, keepdims=True))

if use_detector_score:
    print(img_input_feats.shape, faceness_scores.shape)
    img_input_feats = img_input_feats * faceness_scores[:, np.newaxis]
else:
    img_input_feats = img_input_feats

In [60]:
template_norm_feats, unique_templates = image2template_feature(
    img_input_feats, templates, medias)
stop = timeit.default_timer()
print('Time: %.2f s. ' % (stop - start))

Finish Calculating 0 template features.
Finish Calculating 2000 template features.
Finish Calculating 4000 template features.
Finish Calculating 6000 template features.
Finish Calculating 8000 template features.
Finish Calculating 10000 template features.
Finish Calculating 12000 template features.
Finish Calculating 14000 template features.
Finish Calculating 16000 template features.
Finish Calculating 18000 template features.
Finish Calculating 20000 template features.
Finish Calculating 22000 template features.
Time: 8.82 s. 


In [61]:
template_norm_feats.shape

(23124, 512)

In [62]:
unique_templates.shape

(23124,)

# STEP4: Get Template Similarity Scores

In [63]:
def verification(template_norm_feats=None,
                 unique_templates=None,
                 p1=None,
                 p2=None):
    # ==========================================================
    #         Compute set-to-set Similarity Score.
    # ==========================================================
    template2id = np.zeros((max(unique_templates) + 1, 1), dtype=int)
    for count_template, uqt in enumerate(unique_templates):
        template2id[uqt] = count_template

    score = np.zeros((len(p1),))  # save cosine distance between pairs

    total_pairs = np.array(range(len(p1)))
    batchsize = 100000  # small batchsize instead of all pairs in one batch due to the memory limiation
    sublists = [
        total_pairs[i:i + batchsize] for i in range(0, len(p1), batchsize)
    ]
    total_sublists = len(sublists)
    for c, s in enumerate(sublists):
        feat1 = template_norm_feats[template2id[p1[s]]]
        feat2 = template_norm_feats[template2id[p2[s]]]
        similarity_score = np.sum(feat1 * feat2, -1)
        score[s] = similarity_score.flatten()
        if c % 10 == 0:
            print('Finish {}/{} pairs.'.format(c, total_sublists))
    return score

In [64]:
# =============================================================
# compute verification scores between template pairs.
# =============================================================
start = timeit.default_timer()
score = verification(template_norm_feats, unique_templates, p1, p2)
stop = timeit.default_timer()
print('Time: %.2f s. ' % (stop - start))

# In[ ]:
save_path = "result/"
# save_path = result_dir + '/%s_result' % target

if not os.path.exists(save_path):
    os.makedirs(save_path)
target = f'{head_type}_{backbone_type}'
score_save_file = os.path.join(save_path, "%s.npy" % target.lower())
np.save(score_save_file, score)

Finish 0/157 pairs.
Finish 10/157 pairs.
Finish 20/157 pairs.
Finish 30/157 pairs.
Finish 40/157 pairs.
Finish 50/157 pairs.
Finish 60/157 pairs.
Finish 70/157 pairs.
Finish 80/157 pairs.
Finish 90/157 pairs.
Finish 100/157 pairs.
Finish 110/157 pairs.
Finish 120/157 pairs.
Finish 130/157 pairs.
Finish 140/157 pairs.
Finish 150/157 pairs.
Time: 127.16 s. 


# STEP5: Compute ROC

In [65]:
def compute_ROC(labels, scores, roc_path):
    print('==> compute ROC.')
    import sklearn.metrics as skm
    from scipy import interpolate
    fpr, tpr, thresholds = skm.roc_curve(labels, scores)
    fpr_levels = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1]
    f_interp = interpolate.interp1d(fpr, tpr)
    tpr_at_fpr = [f_interp(x) for x in fpr_levels]
    roc_txt = f'ROC_{head_type}_{backbone_type}.txt'.lower()
    file = open(f'{save_path}{roc_txt}', 'w')
    for (far, tar) in zip(fpr_levels, tpr_at_fpr):
        print('TAR @ FAR = {} : {}'.format(far, tar))
        file.write('TAR @ FAR = {}: {}\n'.format(far, tar))
    file.close()

In [66]:
compute_ROC(label, score, save_path)

==> compute ROC.
TAR @ FAR = 1e-05 : 0.00021201249939685154
TAR @ FAR = 0.0001 : 0.0010062402476287682
TAR @ FAR = 0.001 : 0.002863424860663701
TAR @ FAR = 0.01 : 0.017794140205552998
TAR @ FAR = 0.1 : 0.1323822672188986
